### Historical and CDC loading 
###### database: Insurance
###### table: Policy

In [10]:
%iam_role arn:aws:iam::331504768406:role/service-role/AWSGlueServiceRole
%region us-east-1
%idle_timeout 5
%glue_version 4.0
%worker_type G.1X
%number_of_workers 2


Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Current iam_role is arn:aws:iam::331504768406:role/service-role/AWSGlueServiceRole
iam_role has been set to arn:aws:iam::331504768406:role/service-role/AWSGlueServiceRole.
Previous region: us-east-1
Setting new region to: us-east-1
Region is set to: us-east-1
Current idle_timeout is None minutes.
idle_timeout has been set to 5 minutes.
Setting Glue version to: 4.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 2


In [12]:

%%configure -f
{
    "conf": "spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog",
    "datalake-formats":"delta",
    'enable-auto-scaling': 'false',
    'JOB_NAME': 'glue-job-policy-insurance-full-load', 
    's3_bucket': 's3://jamil-datalake-dev/',
    'start_date': 'cron',
    'final_date': 'cron',
    'environment': 'prd', 
    'file_format': 'csv',
    'reprocess_all': 'False'
}


The following configurations have been updated: {'conf': 'spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog', 'datalake-formats': 'delta', 'enable-auto-scaling': 'false', 'JOB_NAME': 'glue-job-policy-insurance-full-load', 's3_bucket': 's3://jamil-datalake-dev/', 'start_date': 'cron', 'final_date': 'cron', 'environment': 'prd', 'file_format': 'csv', 'reprocess_all': 'False'}


In [1]:
import sys
import boto3
import pyspark.sql.functions as F
from pyspark.context import SparkContext
from pyspark.sql import DataFrame
from pyspark.sql.types import StructType, StructField, StringType
from awsglue.utils import getResolvedOptions
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame
from delta import DeltaTable
from datetime import date, datetime, timedelta
from dateutil.parser import parse
from re import sub

t1 = datetime.now()
glueContext = GlueContext(SparkContext.getOrCreate())
spark = glueContext.spark_session

Trying to create a Glue session for the kernel.
Session Type: etl
Worker Type: G.1X
Number of Workers: 2
Session ID: 52afbd35-dec8-4713-99bc-8de8cae9ee0a
Applying the following default arguments:
--glue_kernel_version 1.0.4
--enable-glue-datacatalog true
--conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog
--datalake-formats delta
--enable-auto-scaling false
--JOB_NAME glue-job-policy-insurance-full-load
--s3_bucket s3://jamil-datalake-dev/
--start_date cron
--final_date cron
--environment prd
--file_format csv
--reprocess_all False
Waiting for session 52afbd35-dec8-4713-99bc-8de8cae9ee0a to get into ready status...
Session 52afbd35-dec8-4713-99bc-8de8cae9ee0a has been created.



In [2]:
# convert a text to a snake case string
def to_snake_case(text):
    return '_'.join(
                    sub('([A-Z][a-z]+)', r' \1',
                    sub('([A-Z]+)', r' \1',
                        text.replace('-', ' ')
                        )).split()
                    ).lower()


In [3]:
def dedup_keys_str(primary_keys: list) -> str:

    condition_list = [f'target.{key} = delta.{key}' for key in primary_keys]

    dedup_str = ' AND '.join(condition_list)
            
    return dedup_str


In [4]:
def table_exists(database, table) -> bool:
    
    exist = spark.sql(f""" select * 
                            from {database}.{table} 
                            limit 1
                        """)
            
    return bool(exist.count() > 0) 


In [5]:
def delta_table_exists(path) -> bool:

    exist = False
    try:
        delta = DeltaTable.forPath(spark, path)
        exist = True
    except:
        exist = False

    return exist    

In [6]:

def is_valid_date(date_ymd) -> bool:

    is_valid = False

    if date_ymd:
        try:
            parse(timestr=date_ymd, yearfirst=True, dayfirst=True)
            is_valid = True
        except:
            is_valid = False
    
    return is_valid


In [7]:
def get_s3_bucket_objects(bucket_name, prefix, start_date=None, final_date=None, file_format='csv') -> list:
      
    s3 = boto3.client('s3')
    file_list = s3.list_objects_v2(Bucket= bucket_name.removeprefix("s3://").removesuffix('/'), 
                                   Prefix= prefix.removeprefix(bucket_name))

    if 'Contents' in file_list:
        content = file_list['Contents']
    else:
        return []


    if 'Key' in content[0]:

        if start_date == None:
            start_date = '2001-01-01' ## all possible dates

        if final_date == None:
            final_date = '2099-12-31'

        start_date = date.fromisoformat(start_date).strftime('%Y%m%d')
        final_date = date.fromisoformat(final_date).strftime('%Y%m%d')
        
        obj_list = []
        for obj in content:
            obj = obj['Key']
            
            if obj.endswith(f".{file_format}"):
                
                if obj[-12:-4] >= start_date and obj[-12:-4] <= final_date: 
                    obj_list.append(bucket_name + obj)
    
        obj_list.sort()

    else:
        return []   
       
    return obj_list



In [8]:
def s3_bucket_exists(bucket_name) -> bool:
    
    s3 = boto3.client('s3')
    obj_list = [obj['Name'] for obj in s3.list_buckets()['Buckets'] ]

    return bucket_name in obj_list



In [9]:
# reading source data file

def read_source(path, schema):
            
    source_df = (spark.read
                    .format("csv")
                    .schema(schema)
                    .option("header", "true")
                    .load(path)
                )

    return source_df


In [10]:
# transforming data and columns

def transform(data_frame):
      
      # apply mapping
      dyf = DynamicFrame.fromDF(data_frame, glueContext, "dyf")

      mappings = [('operation', 'string', 'operation', 'char(1)'), 
                  ('policy_id', 'string', 'policy_id', 'bigint'), 
                  ('expiry_date', 'string', 'expiry_date', 'date'), 
                  ('location_name', 'string', 'location_name', 'string'), 
                  ('state_code', 'string', 'state_code', 'string'), 
                  ('region_name', 'string', 'region_name', 'string'), 
                  ('insured_value', 'string', 'insured_value', 'double'), 
                  ('business_type', 'string', 'business_type', 'string'), 
                  ('earthquake', 'string', 'earth_quake', 'char(1)'), 
                  ('flood', 'string', 'flood', 'char(1)')]
#                  ('update_date', 'string', 'update_date', 'date')]

      dyf = dyf.apply_mapping(mappings)
      data_frame = dyf.toDF()

      target_df = (data_frame
                        .withColumn('file_name',      F.input_file_name())
                        .withColumn('year_month_day', F.expr("substring(file_name, length(file_name) -11, 8)"))
                        .withColumn('year',           F.expr("substring(year_month_day, 1, 4)"))
                        .withColumn('month',          F.expr("substring(year_month_day, 5, 2)"))
                        .withColumn('day',            F.expr("substring(year_month_day, 7, 2)"))
                        .drop('operation')
                        .dropDuplicates()
                  )

      #target_df = target_df.select([F.col(c) for c in target_df.columns])
      return target_df


In [11]:
# writing first time

def historical_load(target_df, path):

    try:
        (target_df.write
            .format('delta')
            .mode('overwrite') 
            .partitionBy(['year', 'month', 'day'])
            .option("overwriteSchema", "true")
            .option("path", path)
            .save()
        )
    except:
        raise ValueError(f"**** Error saving into the bucket {path}")
        

In [12]:
# upsert

def delta_load(delta_df, primary_keys, path):

    try:
        target_df = DeltaTable.forPath(spark, path)
    except:
        raise ValueError('**** Target S3 target folder has not found.')

    try:
        (target_df.alias('target')
                .merge( source    = delta_df.alias('delta'),
                        condition = F.expr(dedup_keys_str(primary_keys)))
                .whenMatchedUpdateAll()
                .whenNotMatchedInsertAll()
        ).execute()
    except:
        raise ValueError(f"**** Error upserting into bucket {path}")


In [13]:
def get_schema():
    schema_fields = [
            ('operation', 'string'),
            ('policy_id', 'string'),
            ('expiry_date', 'string'),
            ('location_name', 'string'),
            ('state_code', 'string'),
            ('region_name', 'string'),
            ('insured_value', 'string'),
            ('business_type', 'string'),
            ('earthquake', 'string'),
            ('flood', 'string')
        ]

    schema = StructType(
                [StructField(field_name, StringType(), True) for field_name, _ in schema_fields]
            )
    
    return schema

In [14]:

def main(args) -> None:
    reprocess_all= (args['reprocess_all'] == 'True') # when reprocess_all = true, all data will be deleted and reloaded
    environment  = args['environment']
    file_format  = args['file_format'] # not nullable
    s3_bucket    = args['s3_bucket']   # not nullable
    start_date   = args['start_date']  # not nullable
    final_date   = args['final_date']  # not nullable
    days_ago     = 0

    ingestion    = 'raw-data'
    catalog      = 'glue-catalog'
    database     = 'insurance_db'
    table_name   = 'policy'
    primary_keys = ['policy_id']

    prefix_full_load = ingestion +'/'+ database +'/'+ table_name + '/full-load/'
    prefix_cdc_load  = ingestion +'/'+ database +'/'+ table_name + '/cdc-load/'

    source_path_full = s3_bucket + prefix_full_load 
    source_path_cdc = s3_bucket + prefix_cdc_load

    target_path      = s3_bucket + catalog +'/'+ database +'/'+ table_name + '/'

    if not s3_bucket_exists(s3_bucket.removeprefix("s3://").removesuffix('/')):
        raise ValueError('**** Bucket name is invalid.')
    
    if (start_date != 'cron' and not is_valid_date(start_date)) or (final_date != 'cron' and not is_valid_date(final_date)):
        raise ValueError('**** Start or final date is invalid.')
    
    if reprocess_all not in [True, False]:
        raise ValueError('**** The parameter reprocess_all must be boolean: (True or False).')       
        
    if environment not in ['dev', 'prd']:
        raise ValueError('**** The parameter environment must be [dev or prd].')

    if reprocess_all: 
        start_date  = '2001-01-01'
        final_date  = date.today().strftime('%Y-%m-%d')
        
        ## TODO: 
         # to backup delta table 
         # to delete delta delta table
    else:
        if start_date == 'cron':
            start_date = (date.today() - timedelta(days=days_ago)).strftime('%Y-%m-%d')
        if final_date == 'cron':
            final_date = start_date


    if delta_table_exists(target_path):
        source_path = source_path_cdc
        qtty_before = spark.read.format('delta').load(target_path).count()
    else: 
        source_path = source_path_full
        qtty_before = 0
            

    #get file names from source path (cdc or full)
    file_list = get_s3_bucket_objects(bucket_name=  s3_bucket, 
                                        prefix=     source_path,
                                        start_date= start_date,
                                        final_date= final_date,
                                        file_format=file_format)

    if not file_list:
        print('No one file for loading.')
        qtty_src   = 0
    else:
        # reading files between start and final dates
        src_df = read_source(file_list, get_schema())
        qtty_src = src_df.count()
        
        # transforming
        final_df = transform(src_df)

        # loading historical or delta data
        if delta_table_exists(target_path):
            print(' >>> Delta loading')
            delta_load(final_df, primary_keys, target_path)
        else:
            print(' >>> Historiccal loading')
            historical_load(final_df, target_path)
        
    #checking data loaded
    delta_df = (spark.read
                    .format('delta')
                    .load(target_path)
                )
    qtty_after = delta_df.count()

    print('Start date  : ', start_date)
    print('Final date  : ', final_date)
    print('Source path :', source_path)
    print('Target path :', target_path)
    print('Qtty in DB  :', qtty_before)
    print('Qtty in file:', qtty_src)
    print('Qtty after  :', qtty_after)



In [15]:

args = getResolvedOptions(sys.argv, ['JOB_NAME', 'reprocess_all', 'file_format', 'environment', 's3_bucket', 'start_date', 'final_date'])

job = Job(glueContext)
job.init(args["JOB_NAME"], args)

try:
    main(args)
except Exception as ex:
    print('Error: ', ex)    

print('Elapsed time: ', datetime.now() - t1)


 >>> Delta loading
Start date  :  2024-01-29
Final date  :  2024-01-29
Source path : s3://jamil-datalake-dev/raw-data/insurance_db/policy/cdc-load/
Target path : s3://jamil-datalake-dev/glue-catalog/insurance_db/policy/
Qtty in DB  : 32
Qtty in file: 6
Qtty after  : 33
Elapsed time:  0:01:31.850433
/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.


In [16]:
job.commit()

'''
src_df = read_source(['s3://jamil-datalake-dev/raw-data/insurance_db/policy/cdc-load/cdc-load-20240124.csv'], '2024-01-24','2024-01-24')
final_df = transform(src_df)
src_df.show()
final_df.show()
'''

"\nsrc_df = read_source(['s3://jamil-datalake-dev/raw-data/insurance_db/policy/cdc-load/cdc-load-20240124.csv'], '2024-01-24','2024-01-24')\nfinal_df = transform(src_df)\nsrc_df.show()\nfinal_df.show()\n"


In [14]:
%stop_session

Stopping session: 52afbd35-dec8-4713-99bc-8de8cae9ee0a
Stopped session.
